# Enhancing LLM Performance with Agent Introspection and Trustworthy LLM's

### 🌀 Hallucination Reduction: Problem Setup
In this notebook, we will use introspective agents to perform critical evaluation of an AI agent's own outputs. This process helps in identifying potential errors or biases, followed by a corrective cycle until a predefined trustworthiness threshold is met.

👌 Correction Mechanisms

The correction mechanism engages the AI agent in iterative refinement of its responses. The refinement process looks at the TLM scores provided by Cleanlab as a 3rd party measure to ensure that the final output is both accurate and reliable. For more information about the research underlying this approach, please refer to the [BSDetector uncertainty estimation paper](https://arxiv.org/pdf/2308.16175).



### Setup

In this notebook, we explore the integration of `llama-index-agent-introspective` package to enhance the performance of language models (LLMs). This approach aims to mitigate hallucinations and improve the trustworthiness of LLM outputs. Below are the sources of inspiration for this approach:
- **Llama Index**: [Llama Index GitHub](https://github.com/jerryjliu/llama_index) and [Introspection](https://docs.llamaindex.ai/en/stable/api_reference/agent/introspective/)
- **Cleanlab**: [Cleanlab TLM](https://tlm.cleanlab.ai/) ***You will need to sign up and get an API key to access this service***
- **CRITIC: Large Language Models Can Self-Correct with Tool-Interactive Critiquing**: [CRITIC Paper](https://arxiv.org/abs/2305.11738)



In [1]:
import os
from dotenv import load_dotenv
from cleanlab_studio import Studio

# You will need to register for an API key
studio = Studio(os.getenv("CLEANLAB_API_KEY"))
tlm = studio.TLM()

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
from typing import Tuple
from llama_index.core.bridge.pydantic import Field
from llama_index.core.tools import FunctionTool

def trustworthy_function_tool(
    text: str = Field(
        default_factory=str,
        description="The text to compute trustworthiness score on.",
    )
) -> Tuple[str, float]:
    """Returns the trustworthiness score."""

    response = tlm.prompt(text)
    return ('trustworthiness', response["trustworthiness_score"])

trustworthy_tool = FunctionTool.from_defaults(
    trustworthy_function_tool,
)

In [4]:
from llama_index.agent.introspective import (
    IntrospectiveAgentWorker, 
    ToolInteractiveReflectionAgentWorker
)
from llama_index.llms.openai import OpenAI
from llama_index.agent.openai import OpenAIAgentWorker
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core import ChatPromptTemplate

def get_introspective_agent_with_tool_interactive_reflection(verbose=True, with_main_worker=False):
    """Helper function to build an introspective agent using tool-interactive reflection."""
    
    def create_critique_agent_worker(verbose):
        return FunctionCallingAgentWorker.from_tools(
            tools=[trustworthy_tool], 
            llm=OpenAI("gpt-3.5-turbo"),
            verbose=verbose
        )
    
    def create_stopping_callable():
        """Function to determine stopping condition for reflection & correction cycles."""
        def stopping_callable(critique_str: str) -> bool:
            return "[PASS]" in critique_str
        return stopping_callable
    
    critique_agent_worker = create_critique_agent_worker(verbose)
    correction_llm = OpenAI("gpt-4-turbo-preview")
    stopping_callable = create_stopping_callable()

    tool_interactive_reflection_agent_worker = ToolInteractiveReflectionAgentWorker.from_defaults(
        critique_agent_worker=critique_agent_worker,
        critique_template=(
            "Please provide a trustworthiness critique for the provided text. "
            "If the trustworthiness score is over 0.87 then write '[PASS]', otherwise write '[FAIL]'. "
            "Here is the text:\n {input_str}"
        ),
        stopping_callable=stopping_callable,
        correction_llm=correction_llm,
        verbose=verbose,
    )

    main_agent_worker = (
        OpenAIAgentWorker.from_tools(tools=[], llm=correction_llm, verbose=True)
        if with_main_worker else None
    )

    introspective_agent_worker = IntrospectiveAgentWorker.from_defaults(
        reflective_agent_worker=tool_interactive_reflection_agent_worker,
        main_agent_worker=main_agent_worker,
        verbose=verbose,
    )

    chat_history = [
        ChatMessage(
            content="You are an assistant that generates improved and more credible versions of potentially untrustworthy, user-supplied text.",
            role=MessageRole.SYSTEM,
        )
    ]

    return introspective_agent_worker.as_agent(chat_history=chat_history, verbose=verbose)

# Instantiate the introspective agent
introspective_agent = get_introspective_agent_with_tool_interactive_reflection(verbose=True)

### 🌍 Real-World Erroneous Examples

Let's now experiment with some Real-World examples of misinformation. I've gathered a potporri of common myths and misconceptions that are often shared as facts. Let's see how our introspective agent can help us identify and correct these errors.


1. > "Everyone should follow a gluten free diet."
2. > "Full-fat products equal weight gain."
3. > "The best way to decrease your sodium intake is to stop using the salt shaker."
4. > "You shouldn't eat anything after 7 p.m. — not even a grape."

In [5]:
inaccurate_text = (
    "Everyone should follow a gluten free diet."
    "Full-fat products equal weight gain."
    "The best way to decrease your sodium intake is to stop using the salt shaker."
    "You shouldn't eat anything after 7 p.m. — not even a grape."
)

response = await introspective_agent.achat(inaccurate_text)

> Running step 4cac6f63-d734-473c-8cfa-ae221fafdadb. Step input: Everyone should follow a gluten free diet.Full-fat products equal weight gain.The best way to decrease your sodium intake is to stop using the salt shaker.You shouldn't eat anything after 7 p.m. — not even a grape.
Added user message to memory: Everyone should follow a gluten free diet.Full-fat products equal weight gain.The best way to decrease your sodium intake is to stop using the salt shaker.You shouldn't eat anything after 7 p.m. — not even a grape.
> Running step dc5e1af7-c6a4-4144-834a-f8b3c01afb88. Step input: Everyone should follow a gluten free diet.Full-fat products equal weight gain.The best way to decrease your sodium intake is to stop using the salt shaker.You shouldn't eat anything after 7 p.m. — not even a grape.
> Running step 62e84e05-f0dc-4145-8a03-8c770eb4e1ac. Step input: Please provide a trustworthiness critique for the provided text. If the trustworthiness score is over 0.87 then write '[PASS]', ot

Exception in callback _SelectorSocketTransport._read_ready()
handle: <Handle _SelectorSocketTransport._read_ready()>
Traceback (most recent call last):
  File "/home/esteban/.pyenv/versions/3.11.0/lib/python3.11/asyncio/events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
RuntimeError: cannot enter context: <_contextvars.Context object at 0x7f3c6e5b7a80> is already entered
Exception in callback _SelectorSocketTransport._read_ready()
handle: <Handle _SelectorSocketTransport._read_ready()>
Traceback (most recent call last):
  File "/home/esteban/.pyenv/versions/3.11.0/lib/python3.11/asyncio/events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
RuntimeError: cannot enter context: <_contextvars.Context object at 0x7f3c6e5b7a80> is already entered


=== Calling Function ===
Calling function: trustworthy_function_tool with args: {"text": "Everyone should follow a gluten free diet. Full-fat products equal weight gain. The best way to decrease your sodium intake is to stop using the salt shaker. You shouldn't eat anything after 7 p.m. \u2014 not even a grape."}


Exception in callback _SelectorSocketTransport._read_ready()
handle: <Handle _SelectorSocketTransport._read_ready()>
Traceback (most recent call last):
  File "/home/esteban/.pyenv/versions/3.11.0/lib/python3.11/asyncio/events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
RuntimeError: cannot enter context: <_contextvars.Context object at 0x7f3c6e5b7a80> is already entered
Exception in callback _SelectorSocketTransport._read_ready()
handle: <Handle _SelectorSocketTransport._read_ready()>
Traceback (most recent call last):
  File "/home/esteban/.pyenv/versions/3.11.0/lib/python3.11/asyncio/events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
RuntimeError: cannot enter context: <_contextvars.Context object at 0x7f3c6e5b7a80> is already entered


=== Function Output ===
('trustworthiness', 0.8608914283889307)
> Running step 37138136-c471-4be3-8d2d-969560b4bfcb. Step input: None
=== LLM Response ===
[FAIL]
Critique: [FAIL]
Correction: Everyone should follow a balanced diet, not necessarily gluten-free unless medically required. Full-fat products do not necessarily lead to weight gain; overall calorie intake and expenditure determine weight changes. The best way to decrease your sodium intake is to choose fresh, unprocessed foods and to be mindful of the sodium content in packaged foods. It's not about the specific time you eat, but rather the total calorie intake and quality of food throughout the day that matters.
> Running step 065ccef7-bdee-4f8d-bf24-2fb5f30fb550. Step input: None
> Running step a5a7d480-78ac-4737-badb-6f697f46e960. Step input: Please provide a trustworthiness critique for the provided text. If the trustworthiness score is over 0.87 then write '[PASS]', otherwise write '[FAIL]'. Here is the text:
 Everyone sh

Exception in callback _SelectorSocketTransport._read_ready()
handle: <Handle _SelectorSocketTransport._read_ready()>
Traceback (most recent call last):
  File "/home/esteban/.pyenv/versions/3.11.0/lib/python3.11/asyncio/events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
RuntimeError: cannot enter context: <_contextvars.Context object at 0x7f3c6e5b7a80> is already entered
Exception in callback _SelectorSocketTransport._read_ready()
handle: <Handle _SelectorSocketTransport._read_ready()>
Traceback (most recent call last):
  File "/home/esteban/.pyenv/versions/3.11.0/lib/python3.11/asyncio/events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
RuntimeError: cannot enter context: <_contextvars.Context object at 0x7f3c6e5b7a80> is already entered


=== Calling Function ===
Calling function: trustworthy_function_tool with args: {"text": "Everyone should follow a balanced diet, not necessarily gluten-free unless medically required. Full-fat products do not necessarily lead to weight gain; overall calorie intake and expenditure determine weight changes. The best way to decrease your sodium intake is to choose fresh, unprocessed foods and to be mindful of the sodium content in packaged foods. It's not about the specific time you eat, but rather the total calorie intake and quality of food throughout the day that matters."}


Exception in callback _SelectorSocketTransport._read_ready()
handle: <Handle _SelectorSocketTransport._read_ready()>
Traceback (most recent call last):
  File "/home/esteban/.pyenv/versions/3.11.0/lib/python3.11/asyncio/events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
RuntimeError: cannot enter context: <_contextvars.Context object at 0x7f3c6e5b7a80> is already entered
Exception in callback _SelectorSocketTransport._read_ready()
handle: <Handle _SelectorSocketTransport._read_ready()>
Traceback (most recent call last):
  File "/home/esteban/.pyenv/versions/3.11.0/lib/python3.11/asyncio/events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
RuntimeError: cannot enter context: <_contextvars.Context object at 0x7f3c6e5b7a80> is already entered


=== Function Output ===
('trustworthiness', 0.9092435788290626)
> Running step 50408595-88be-45cd-87f7-6d17d1a02df3. Step input: None
=== LLM Response ===
[PASS] The trustworthiness score for the provided text is 0.909, which is over 0.87.
Critique: [PASS] The trustworthiness score for the provided text is 0.909, which is over 0.87.


In [6]:
response.response

"Everyone should follow a balanced diet, not necessarily gluten-free unless medically required. Full-fat products do not necessarily lead to weight gain; overall calorie intake and expenditure determine weight changes. The best way to decrease your sodium intake is to choose fresh, unprocessed foods and to be mindful of the sodium content in packaged foods. It's not about the specific time you eat, but rather the total calorie intake and quality of food throughout the day that matters."

In [7]:
response.sources

[ToolOutput(content="('trustworthiness', 0.8608914283889307)", tool_name='trustworthy_function_tool', raw_input={'args': ("Everyone should follow a gluten free diet. Full-fat products equal weight gain. The best way to decrease your sodium intake is to stop using the salt shaker. You shouldn't eat anything after 7 p.m. — not even a grape.",), 'kwargs': {}}, raw_output=('trustworthiness', 0.8608914283889307), is_error=False),
 ToolOutput(content="('trustworthiness', 0.9092435788290626)", tool_name='trustworthy_function_tool', raw_input={'args': ("Everyone should follow a balanced diet, not necessarily gluten-free unless medically required. Full-fat products do not necessarily lead to weight gain; overall calorie intake and expenditure determine weight changes. The best way to decrease your sodium intake is to choose fresh, unprocessed foods and to be mindful of the sodium content in packaged foods. It's not about the specific time you eat, but rather the total calorie intake and quality

In [9]:
for msg in introspective_agent.chat_history:
    print(str(msg))
    print()

system: You are an assistant that generates improved and more credible versions of potentially untrustworthy, user-supplied text.

user: Everyone should follow a gluten free diet.Full-fat products equal weight gain.The best way to decrease your sodium intake is to stop using the salt shaker.You shouldn't eat anything after 7 p.m. — not even a grape.

assistant: Everyone should follow a balanced diet, not necessarily gluten-free unless medically required. Full-fat products do not necessarily lead to weight gain; overall calorie intake and expenditure determine weight changes. The best way to decrease your sodium intake is to choose fresh, unprocessed foods and to be mindful of the sodium content in packaged foods. It's not about the specific time you eat, but rather the total calorie intake and quality of food throughout the day that matters.



## Results

The results demonstrate a marked reduction in erroneous outputs and an overall increase in response quality. This approach enhances the reliability and trustworthiness of the generated text.

## Future Directions

There are several potential directions for further research and development in this area:

1. Expand to Other Domains: Apply this reflection-correction mechanism to other sensitive fields such as healthcare, finance, and legal domains to improve the accuracy and reliability of LLM outputs.
2. Real-Time Applications: Develop real-time applications where this technique can be used to provide on-the-fly corrections in conversational AI systems.
3. Automated Feedback Loops: Integrate automated feedback loops where the LLM continuously learns from its corrections and improves over time without human intervention.
4. Enhanced Stopping Criteria: Experiment with more sophisticated stopping criteria that can better determine when the reflection-correction cycle should end.

By pursuing these directions, we can further enhance the robustness and trustworthiness of AI systems, making them more suitable for a wide range of applications.